# Post-processing of output from CREG12.L75-REF08 and CREG12.L75-FUT08

This notebook describes how output from [CREG12.L75-REF08 (Talandier and Lique, 2021)](https://doi.org/10.5281/zenodo.5789520) and [CREG12.L75-FUT08 (Talandier and Lique, 2025)](https://zenodo.org/records/15657723) are post-processed with nco and CDFTOOLS to prepare the data for plotting the figures in Rieck et al. (2025).

First define some variables

```
REF08=$WORK/CREG12.L75-REF08/
FUT08=$WORK/CREG12.L75-FUT08/  
CDF=$WORK/src/CDFTOOLS/
```

We choose 1996-2015 for REF and 2051-2070 for FUT, *i.e.* 20 years for each simulation.

## Compute diagnostics from the 5d output

### Kinetic Energy

```
cd $REF08
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in ${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};                
        fileK=${fileU/gridU/KE};         
        $CDF/cdfeke -u ${fileU} -v ${fileV} -t ${fileT} -mke -nc4 -o $WORK/CREG12.L75-REF08-KE/${fileK};       
        ncrename -O -v vomke,voke $WORK/CREG12.L75-REF08-KE/${fileK} $WORK/CREG12.L75-REF08-KE/${fileK};            
    done; 
done

cd $FUT08
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in ${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};               
        fileK=${fileU/gridU/KE};         
        $CDF/cdfeke -u ${fileU} -v ${fileV} -t ${fileT} -mke -nc4 -o $WORK/CREG12.L75-FUT08-KE/${fileK};       
        ncrename -O -v vomke,voke $WORK/CREG12.L75-FUT08-KE/${fileK} $WORK/CREG12.L75-FUT08-KE/${fileK};     
    done; 
done
```

### Richardson Number

```
cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};         
        fileR=$(echo ${fileU/gridU/RI} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdfrichardson -t ${fileT} -u ${fileU} -v ${fileV} -o $WORK/CREG12.L75-REF08-RI/${fileR};              
    done; 
done

cd $WORK
for yyyy in {2051..2059}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};         
        fileT=${fileU/gridU/gridT};         
        fileR=$(echo ${fileU/gridU/RI} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdfrichardson -t ${fileT} -u ${fileU} -v ${fileV} -o $WORK/CREG12.L75-FUT08-RI/${fileR};              
    done; 
done
```

### Buoyancy

```
cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileT in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridT.nc; do         
        fileS=${fileT};               
        fileB=$(echo ${fileT/gridT/buoy} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdf_buoyancy -t ${fileT} -s ${fileT} -mh $WORK/mesh_hgr.nc -mz $WORK/mesh_zgr.nc -bathy $WORK/mesh_zgr.nc -mask $WORK/mask.nc -o $WORK/CREG12.L75-REF08-buoy/${fileB}; 
    done; 
done

cd $WORK
for yyyy in {2051..2059}; do     
    echo ${yyyy};     
    for fileT in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridT.nc; do         
        fileS=${fileT};               
        fileB=$(echo ${fileT/gridT/buoy} | rev | cut -d'/' -f-2 | rev);                 
        $CDF/cdf_buoyancy -t ${fileT} -s ${fileT} -mh $WORK/mesh_hgr.nc -mz $WORK/mesh_zgr.nc -bathy $WORK/mesh_zgr.nc -mask $WORK/mask.nc -o $WORK/CREG12.L75-FUT08-buoy/${fileB};       
    done; 
done
```

## Calculate climatological seasonal cycles of U, V, W and bouyancy
need to do that first to calculate $\text{EKE}$ and $w'b'$ afterwards. Once these calculated, we can compute the seasonal cycles of the rest all together.

### $u$ and $v$

```
cd $WORK/CREG12.L75-REF08-UV_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v vozocrtx -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridU.nc U_tmp${step}.nc
    ncra -O -4 -v vomecrty -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridV.nc V_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-RI/1996/CREG12.L75-REF08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    tmp2=$(echo ${tmp/RI/gridUclim})
    fileU=${tmp2/y1996/}
    fileV=${fileU/gridUclim/gridVclim}
    mv U_tmp${K}.nc ${fileU}
    mv V_tmp${K}.nc ${fileV}
    k=$(($k+1))
done

cd $WORK/CREG12.L75-FUT08-UV_clim
for ffivedstep in {0..72}; 
    do fstep=`printf "%02d" ${ffivedstep}`
    echo ${fstep}; 
    ncra -O -4 -v vozocrtx -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridU.nc fU_tmp${step}.nc
    ncra -O -4 -v vomecrty -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridV.nc fV_tmp${step}.nc
done

fk=0
for ffileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-RI/2051/CREG12.L75-FUT08_y*.5d_RI.nc;
    do fK=`printf "%02d" ${fk}`
    ftmp=$(echo ${ffileR} | rev | cut -d'/' -f-1 | rev)
    ftmp2=$(echo ${ftmp/RI/gridUclim})
    ffileU=${ftmp2/y2051/}
    ffileV=${ffileU/gridUclim/gridVclim}
    mv fU_tmp${fK}.nc ${ffileU}
    mv fV_tmp${fK}.nc ${ffileV}
    fk=$(($fk+1))
done
```

### $w$ 

```
cd $WORK/CREG12.L75-REF08-W_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v vovecrtz -d time_counter,${fivedstep},,73 $WORK/CREG12.L75-REF08-gridW/????/CREG12.L75-REF08_y*.5d_gridW.nc W_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-RI/1996/CREG12.L75-REF08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    tmp2=$(echo ${tmp/RI/gridWclim})
    fileW=${tmp2/y1996/}
    mv W_tmp${K}.nc ${fileW}
    k=$(($k+1))
done

cd $WORK/CREG12.L75-FUT08-W_clim
for ffivedstep in {0..72}; 
    do fstep=`printf "%02d" ${ffivedstep}`
    echo ${fstep}; 
    ncra -O -4 -v vovecrtz -d time_counter,${ffivedstep},,73 $WORK/CREG12.L75-FUT08-gridW/????/CREG12.L75-FUT08_y*.5d_gridW.nc fW_tmp${fstep}.nc
done

fk=0
for ffileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-RI/2051/CREG12.L75-FUT08_y*.5d_RI.nc;
    do fK=`printf "%02d" ${fk}`
    ftmp=$(echo ${ffileR} | rev | cut -d'/' -f-1 | rev)
    ftmp2=$(echo ${ftmp/RI/gridWclim})
    ffileW=${ftmp2/y2051/}
    mv fW_tmp${fK}.nc ${ffileW}
    fk=$(($fk+1))
done
```

### Buoyancy

```
cd $WORK/CREG12.L75-REF08-BUOY_clim
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step}; 
    ncra -O -4 -v cv_bn2 -d time_counter,${fivedstep},,73 $WORK/CREG12.L75-REF08-buoy/????/CREG12.L75-REF08_y*.5d_buoy.nc B_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-RI/1996/CREG12.L75-REF08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    tmp2=$(echo ${tmp/RI/BUOYclim})
    fileB=${tmp2/y1996/}
    mv B_tmp${K}.nc ${fileB}
    k=$(($k+1))
done

cd $WORK/CREG12.L75-FUT08-BUOY_clim
for ffivedstep in {0..72}; 
    do fstep=`printf "%02d" ${ffivedstep}`
    echo ${fstep}; 
    ncra -O -4 -v cv_bn2 -d time_counter,${ffivedstep},,73 $WORK/CREG12.L75-FUT08-buoy/????/CREG12.L75-FUT08_y*.5d_buoy.nc fB_tmp${fstep}.nc
done

fk=0
for ffileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-RI/2051/CREG12.L75-FUT08_y*.5d_RI.nc;
    do fK=`printf "%02d" ${fk}`
    ftmp=$(echo ${ffileR} | rev | cut -d'/' -f-1 | rev)
    ftmp2=$(echo ${ftmp/RI/BUOYclim})
    ffileB=${ftmp2/y2051/}
    mv fB_tmp${fK}.nc ${ffileB}
    fk=$(($fk+1))
done
```

## Calculate "prime" quantities as anomalies to the climatlogy
We compute $\overline{u}$, $\overline{v}$, $u'$, $v'$ to get $\text{MKE}$ and $\text{EKE}$, as well as $w'$ and $b'$  
We include interannual variations in $\overline{\cdot}$ such that the anomaly $\cdot ' = \text{total} - (\text{annual mean} - \text{long-term mean} + \text{climatology}))$.

### First compute long term mean

```
cd $WORK
ncra -O -4 -v vozocrtx $REF08M/????/CREG12.L75-REF08_y????.5d_gridU.nc CREG12.L75-REF08-UV_20yM/CREG12.L75-REF08_y1996-2015_gridU.nc
ncra -O -4 -v vomecrty $REF08M/????/CREG12.L75-REF08_y????.5d_gridV.nc CREG12.L75-REF08-UV_20yM/CREG12.L75-REF08_y1996-2015_gridV.nc

cd $WORK
ncra -O -4 -v vovecrtz CREG12.L75-REF08-gridW/????/CREG12.L75-REF08_y????m??d??.5d_gridW.nc CREG12.L75-REF08-gridW_20yM/CREG12.L75-REF08_y1996-2015_gridW.nc
ncra -O -4 -v cv_bn2 CREG12.L75-REF08-buoy/????/CREG12.L75-REF08_y????m??d??.5d_buoy.nc CREG12.L75-REF08-BUOY_20yM/CREG12.L75-REF08_y1996-2015_BUOY.nc

cd $WORK
ncra -O -4 -v vozocrtx $FUT08M/2???/CREG12.L75-FUT08_y????.5d_gridU.nc CREG12.L75-FUT08-UV_20yM/CREG12.L75-FUT08_y2051-2070_gridU.nc
ncra -O -4 -v vomecrty $FUT08M/2???/CREG12.L75-FUT08_y????.5d_gridV.nc CREG12.L75-FUT08-UV_20yM/CREG12.L75-FUT08_y2051-2070_gridV.nc

cd $WORK
ncra -O -4 -v vovecrtz CREG12.L75-FUT08-gridW/????/CREG12.L75-FUT08_y????m??d??.5d_gridW.nc CREG12.L75-FUT08-gridW_20yM/CREG12.L75-FUT08_y2051-2070_gridW.nc
ncra -O -4 -v cv_bn2 CREG12.L75-FUT08-buoy/????/CREG12.L75-FUT08_y????m??d??.5d_buoy.nc CREG12.L75-FUT08-BUOY_20yM/CREG12.L75-FUT08_y2051-2070_BUOY.nc
```

### Now annual_mean - long term mean

```
cd $WORK
fileUm=CREG12.L75-REF08-UV_20yM/CREG12.L75-REF08_y1996-2015_gridU.nc
fileVm=${fileUm/gridU/gridV}
for yyyy in {1996..2015}; do     
    echo ${yyyy};
    fileUa=$REF08M/${yyyy}/CREG12.L75-REF08_y${yyyy}.5d_gridU.nc
    fileVa=${fileUa/gridU/gridV}
    fileUam=CREG12.L75-REF08-UV_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_gridU_altMEAN.nc
    fileVam=${fileUam/gridU/gridV}
    ncrename -O -v vozocrtx,vozocrtxAnnualMEAN ${fileUa} ${fileUam}
    ncrename -O -v vomecrty,vomecrtyAnnualMEAN ${fileVa} ${fileVam}
    ncks -A -v vozocrtx -4 ${fileUm} ${fileUam} # here, vozocrtx is now the long term mean
    ncks -A -v vomecrty -4 ${fileVm} ${fileVam}
    ncap2 -O -4 -s 'vozocrtxAM=vozocrtxAnnualMEAN-vozocrtx' ${fileUam} ${fileUam}
    ncap2 -O -4 -s 'vomecrtyAM=vomecrtyAnnualMEAN-vomecrty' ${fileVam} ${fileVam}
done

cd $WORK
fileWm=CREG12.L75-REF08-gridW_20yM/CREG12.L75-REF08_y1996-2015_gridW.nc
fileBm=CREG12.L75-REF08-BUOY_20yM/CREG12.L75-REF08_y1996-2015_BUOY.nc
for yyyy in {1996..2015}; do     
    echo ${yyyy};
    fileWa=tmpW_y${yyyy}.nc
    fileBa=tmpB_y${yyyy}.nc
    ncra -O -4 -v vovecrtz CREG12.L75-REF08-gridW/${yyyy}/CREG12.L75-REF08_y${yyyy}m??d??.5d_gridW.nc ${fileWa}
    ncra -O -4 -v cv_bn2 CREG12.L75-REF08-buoy/${yyyy}/CREG12.L75-REF08_y${yyyy}m??d??.5d_buoy.nc ${fileBa}
    fileWam=CREG12.L75-REF08-gridW_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_gridW_altMEAN.nc
    fileBam=CREG12.L75-REF08-BUOY_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_BUOY_altMEAN.nc
    ncrename -O -v vovecrtz,vovecrtzAnnualMEAN ${fileWa} ${fileWam}
    ncrename -O -v cv_bn2,cv_bn2AnnualMEAN ${fileBa} ${fileBam}
    ncks -A -v vovecrtz -4 ${fileWm} ${fileWam} # here, vovecrtz is now the long term mean
    ncks -A -v cv_bn2 -4 ${fileBm} ${fileBam}
    ncap2 -O -4 -s 'vovecrtzAM=vovecrtzAnnualMEAN-vovecrtz' ${fileWam} ${fileWam}
    ncap2 -O -4 -s 'cv_bn2AM=cv_bn2AnnualMEAN-cv_bn2' ${fileBam} ${fileBam}
done

cd $WORK
fileUm=CREG12.L75-FUT08-UV_20yM/CREG12.L75-FUT08_y2051-2070_gridU.nc
fileVm=${fileUm/gridU/gridV}
for yyyy in {2051..2070}; do     
    echo ${yyyy};
    fileUa=$FUT08M/${yyyy}/CREG12.L75-FUT08_y${yyyy}.5d_gridU.nc
    fileVa=${fileUa/gridU/gridV}
    fileUam=CREG12.L75-FUT08-UV_annual-longtermMEAN/CREG12.L75-FUT08_y${yyyy}_gridU_altMEAN.nc
    fileVam=${fileUam/gridU/gridV}
    ncrename -O -v vozocrtx,vozocrtxAnnualMEAN ${fileUa} ${fileUam}
    ncrename -O -v vomecrty,vomecrtyAnnualMEAN ${fileVa} ${fileVam}
    ncks -A -v vozocrtx -4 ${fileUm} ${fileUam} # here, vozocrtx is now the long term mean
    ncks -A -v vomecrty -4 ${fileVm} ${fileVam}
    ncap2 -O -4 -s 'vozocrtxAM=vozocrtxAnnualMEAN-vozocrtx' ${fileUam} ${fileUam}
    ncap2 -O -4 -s 'vomecrtyAM=vomecrtyAnnualMEAN-vomecrty' ${fileVam} ${fileVam}
done

cd $WORK
ffileWm=CREG12.L75-FUT08-gridW_20yM/CREG12.L75-FUT08_y2051-2070_gridW.nc
ffileBm=CREG12.L75-FUT08-BUOY_20yM/CREG12.L75-FUT08_y2051-2070_BUOY.nc
for fyyyy in {2051..2070}; do     
    echo ${fyyyy};
    ffileWa=ftmpW_y${fyyyy}.nc
    ffileBa=ftmpB_y${fyyyy}.nc
    ncra -O -4 -v vovecrtz CREG12.L75-FUT08-gridW/${fyyyy}/CREG12.L75-FUT08_y${fyyyy}m??d??.5d_gridW.nc ${ffileWa}
    ncra -O -4 -v cv_bn2 CREG12.L75-FUT08-buoy/${fyyyy}/CREG12.L75-FUT08_y${fyyyy}m??d??.5d_buoy.nc ${ffileBa}
    ffileWam=CREG12.L75-FUT08-gridW_annual-longtermMEAN/CREG12.L75-FUT08_y${fyyyy}_gridW_altMEAN.nc
    ffileBam=CREG12.L75-FUT08-BUOY_annual-longtermMEAN/CREG12.L75-FUT08_y${fyyyy}_BUOY_altMEAN.nc
    ncrename -O -v vovecrtz,vovecrtzAnnualMEAN ${ffileWa} ${ffileWam}
    ncrename -O -v cv_bn2,cv_bn2AnnualMEAN ${ffileBa} ${ffileBam}
    ncks -A -v vovecrtz -4 ${ffileWm} ${ffileWam} # here, vovecrtz is now the long term mean
    ncks -A -v cv_bn2 -4 ${ffileBm} ${ffileBam}
    ncap2 -O -4 -s 'vovecrtzAM=vovecrtzAnnualMEAN-vovecrtz' ${ffileWam} ${ffileWam}
    ncap2 -O -4 -s 'cv_bn2AM=cv_bn2AnnualMEAN-cv_bn2' ${ffileBam} ${ffileBam}
done
```

### Finally,  full - (annual minus long term + climatology)

```
cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUcTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVcTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUc=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileUcTMP | rev | cut -d'm' -f-2 | rev)
        fileVc=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileVcTMP | rev | cut -d'm' -f-2 | rev)
        fileUm=CREG12.L75-REF08-UV_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_gridU_altMEAN.nc
        fileVm=${fileUm/gridU/gridV}
        fileE=$(echo ${fileU/gridU/EKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxCLIM ${fileUc} tmpU.nc
        ncrename -O -v vomecrty,vomecrtyCLIM ${fileVc} tmpV.nc
        ncks -A -v vozocrtxAM -4 ${fileUm} tmpU.nc
        ncks -A -v vomecrtyAM -4 ${fileVm} tmpV.nc
        ncks -A -v vozocrtx -4 ${fileU} tmpU.nc
        ncks -A -v vomecrty -4 ${fileV} tmpV.nc
        ncap2 -O -4 -s 'vozocrtxPRIME=vozocrtx-(vozocrtxAM+vozocrtxCLIM)' tmpU.nc tmpU.nc
        ncap2 -O -4 -s 'vomecrtyPRIME=vomecrty-(vomecrtyAM+vomecrtyCLIM)' tmpV.nc tmpV.nc
        ncks -O -4 -v vozocrtxPRIME tmpU.nc tmpU.nc
        ncks -O -4 -v vomecrtyPRIME tmpV.nc tmpV.nc
        ncrename -O -v vozocrtxPRIME,vozocrtx tmpU.nc tmpU.nc
        ncrename -O -v vomecrtyPRIME,vomecrty tmpV.nc tmpV.nc
        $CDF/cdfeke -u tmpU.nc -v tmpV.nc -t ${fileT} -mke -o tmpE.nc
        ncks -O -4 -L 1 tmpE.nc tmpE.nc
        ncrename -O -v .vomke,voeke tmpE.nc CREG12.L75-REF08-EKE/${fileE}
    done; 
done

cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileU in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUcTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVcTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUc=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileUcTMP | rev | cut -d'm' -f-2 | rev)
        fileVc=CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m$(echo $fileVcTMP | rev | cut -d'm' -f-2 | rev)
        fileUm=CREG12.L75-REF08-UV_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_gridU_altMEAN.nc
        fileVm=${fileUm/gridU/gridV}
        fileM=$(echo ${fileU/gridU/MKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxCLIM ${fileUc} tmpU.nc
        ncrename -O -v vomecrty,vomecrtyCLIM ${fileVc} tmpV.nc
        ncks -A -v vozocrtxAM -4 ${fileUm} tmpU.nc
        ncks -A -v vomecrtyAM -4 ${fileVm} tmpV.nc
        ncap2 -O -4 -s 'vozocrtx=vozocrtxAM+vozocrtxCLIM' tmpU.nc tmpU.nc
        ncap2 -O -4 -s 'vomecrty=vomecrtyAM+vomecrtyCLIM' tmpV.nc tmpV.nc
        $CDF/cdfeke -u tmpU.nc -v tmpV.nc -t ${fileT} -mke -o CREG12.L75-REF08-MKE/${fileM}
        ncks -A -v time_counter -4 -L 1 ${fileT} CREG12.L75-REF08-MKE/${fileM}
    done
done

cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileW in CREG12.L75-REF08-gridW/${yyyy}/CREG12.L75-REF08_y${yyyy}m??d??.5d_gridW.nc; do         
        fileWcTMP=$(echo ${fileW/_gridW/_gridWclim} | rev | cut -d'/' -f-1 | rev)
        fileWpTMP=$(echo ${fileW/_gridW/_gridWprime} | rev | cut -d'/' -f-1 | rev)
        fileWc=CREG12.L75-REF08-W_clim/CREG12.L75-REF08_m$(echo $fileWcTMP | rev | cut -d'm' -f-2 | rev)
        fileWm=CREG12.L75-REF08-gridW_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_gridW_altMEAN.nc
        fileWp=CREG12.L75-REF08-gridW_prime/${fileWpTMP};
        ncrename -O -v vovecrtz,vovecrtzCLIM ${fileWc} tmpW.nc
        ncks -A -v vovecrtzAM -4 ${fileWm} tmpW.nc
        ncks -A -v vovecrtz -4 ${fileW} tmpW.nc
        ncap2 -O -4 -s 'vovecrtzPRIME=vovecrtz-(vovecrtzAM+vovecrtzCLIM)' tmpW.nc tmpW.nc
        ncks -O -4 -v vovecrtzPRIME tmpW.nc tmpW.nc
        ncrename -O -v vovecrtzPRIME,vovecrtz tmpW.nc ${fileWp}
    done; 
done

cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};   
    for fileB in CREG12.L75-REF08-buoy/${yyyy}/CREG12.L75-REF08_y${yyyy}m??d??.5d_buoy.nc; do
        fileBcTMP=$(echo ${fileB/_buoy/_BUOYclim} | rev | cut -d'/' -f-1 | rev)
        fileBpTMP=$(echo ${fileB/_buoy/_BUOYprime} | rev | cut -d'/' -f-1 | rev)
        fileBc=CREG12.L75-REF08-BUOY_clim/CREG12.L75-REF08_m$(echo $fileBcTMP | rev | cut -d'm' -f-2 | rev)
        fileBm=CREG12.L75-REF08-BUOY_annual-longtermMEAN/CREG12.L75-REF08_y${yyyy}_BUOY_altMEAN.nc;
        fileBp=CREG12.L75-REF08-BUOY_prime/${fileBpTMP};
        ncrename -O -v cv_bn2,cv_bn2CLIM ${fileBc} tmpB.nc
        ncks -A -v cv_bn2AM -4 ${fileBm} tmpB.nc
        ncks -A -v cv_bn2 -4 ${fileB} tmpB.nc
        ncap2 -O -4 -s 'cv_bn2PRIME=cv_bn2-(cv_bn2AM+cv_bn2CLIM)' tmpB.nc tmpB.nc
        ncks -O -4 -v cv_bn2PRIME tmpB.nc tmpB.nc
        ncrename -O -v cv_bn2PRIME,cv_bn2 tmpB.nc ${fileBp}
    done; 
done

cd $WORK
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUcTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVcTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUc=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileUcTMP | rev | cut -d'm' -f-2 | rev)
        fileVc=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileVcTMP | rev | cut -d'm' -f-2 | rev)
        fileUm=CREG12.L75-FUT08-UV_annual-longtermMEAN/CREG12.L75-FUT08_y${yyyy}_gridU_altMEAN.nc
        fileVm=${fileUm/gridU/gridV}
        fileE=$(echo ${fileU/gridU/EKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxCLIM ${fileUc} tmpUf.nc
        ncrename -O -v vomecrty,vomecrtyCLIM ${fileVc} tmpVf.nc
        ncks -A -v vozocrtxAM -4 ${fileUm} tmpUf.nc
        ncks -A -v vomecrtyAM -4 ${fileVm} tmpVf.nc
        ncks -A -v vozocrtx -4 ${fileU} tmpUf.nc
        ncks -A -v vomecrty -4 ${fileV} tmpVf.nc
        ncap2 -O -4 -s 'vozocrtxPRIME=vozocrtx-(vozocrtxAM+vozocrtxCLIM)' tmpUf.nc tmpUf.nc
        ncap2 -O -4 -s 'vomecrtyPRIME=vomecrty-(vomecrtyAM+vomecrtyCLIM)' tmpVf.nc tmpVf.nc
        ncks -O -4 -v vozocrtxPRIME tmpUf.nc tmpUf.nc
        ncks -O -4 -v vomecrtyPRIME tmpVf.nc tmpVf.nc
        ncrename -O -v vozocrtxPRIME,vozocrtx tmpUf.nc tmpUf.nc
        ncrename -O -v vomecrtyPRIME,vomecrty tmpVf.nc tmpVf.nc
        $CDF/cdfeke -u tmpUf.nc -v tmpVf.nc -t ${fileT} -mke -o tmpEf.nc
        ncks -O -4 -L 1 tmpEf.nc tmpEf.nc
        ncrename -O -v .vomke,voeke tmpEf.nc CREG12.L75-FUT08-EKE/${fileE}
    done; 
done

cd $WORK
for yyyy in {2051..2070}; do     
    echo ${yyyy};     
    for fileU in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridU.nc; do         
        fileV=${fileU/gridU/gridV};
        fileT=${fileU/gridU/gridT}; 
        fileUcTMP=$(echo ${fileU/gridU/gridUclim} | rev | cut -d'/' -f-1 | rev)
        fileVcTMP=$(echo ${fileV/gridV/gridVclim} | rev | cut -d'/' -f-1 | rev)
        fileUc=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileUcTMP | rev | cut -d'm' -f-2 | rev)
        fileVc=CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m$(echo $fileVcTMP | rev | cut -d'm' -f-2 | rev)
        fileUm=CREG12.L75-FUT08-UV_annual-longtermMEAN/CREG12.L75-FUT08_y${yyyy}_gridU_altMEAN.nc
        fileVm=${fileUm/gridU/gridV}
        fileM=$(echo ${fileU/gridU/MKE} | rev | cut -d'/' -f-2 | rev);
        ncrename -O -v vozocrtx,vozocrtxCLIM ${fileUc} tmpUf.nc
        ncrename -O -v vomecrty,vomecrtyCLIM ${fileVc} tmpVf.nc
        ncks -A -v vozocrtxAM -4 ${fileUm} tmpUf.nc
        ncks -A -v vomecrtyAM -4 ${fileVm} tmpVf.nc
        ncap2 -O -4 -s 'vozocrtx=vozocrtxAM+vozocrtxCLIM' tmpUf.nc tmpUf.nc
        ncap2 -O -4 -s 'vomecrty=vomecrtyAM+vomecrtyCLIM' tmpVf.nc tmpVf.nc
        $CDF/cdfeke -u tmpUf.nc -v tmpVf.nc -t ${fileT} -mke -o CREG12.L75-FUT08-MKE/${fileM}
        ncks -A -v time_counter -4 -L 1 ${fileT} CREG12.L75-FUT08-MKE/${fileM}
    done
done

cd $WORK
for fyyyy in {2051..2070}; do     
    echo ${fyyyy};     
    for ffileW in CREG12.L75-FUT08-gridW/${fyyyy}/CREG12.L75-FUT08_y${fyyyy}m??d??.5d_gridW.nc; do         
        ffileWcTMP=$(echo ${ffileW/_gridW/_gridWclim} | rev | cut -d'/' -f-1 | rev)
        ffileWpTMP=$(echo ${ffileW/_gridW/_gridWprime} | rev | cut -d'/' -f-1 | rev)
        ffileWc=CREG12.L75-FUT08-W_clim/CREG12.L75-FUT08_m$(echo $ffileWcTMP | rev | cut -d'm' -f-2 | rev)
        ffileWm=CREG12.L75-FUT08-gridW_annual-longtermMEAN/CREG12.L75-FUT08_y${fyyyy}_gridW_altMEAN.nc
        ffileWp=CREG12.L75-FUT08-gridW_prime/${ffileWpTMP};
        ncrename -O -v vovecrtz,vovecrtzCLIM ${ffileWc} ftmpW.nc
        ncks -A -v vovecrtzAM -4 ${ffileWm} ftmpW.nc
        ncks -A -v vovecrtz -4 ${ffileW} ftmpW.nc
        ncap2 -O -4 -s 'vovecrtzPRIME=vovecrtz-(vovecrtzAM+vovecrtzCLIM)' ftmpW.nc ftmpW.nc
        ncks -O -4 -v vovecrtzPRIME ftmpW.nc ftmpW.nc
        ncrename -O -v vovecrtzPRIME,vovecrtz ftmpW.nc ${ffileWp}
    done; 
done

cd $WORK
for fyyyy in {2051..2070}; do     
    echo ${fyyyy};   
    for ffileB in CREG12.L75-FUT08-buoy/${fyyyy}/CREG12.L75-FUT08_y${fyyyy}m??d??.5d_buoy.nc; do
        ffileBcTMP=$(echo ${ffileB/_buoy/_BUOYclim} | rev | cut -d'/' -f-1 | rev)
        ffileBpTMP=$(echo ${ffileB/_buoy/_BUOYprime} | rev | cut -d'/' -f-1 | rev)
        ffileBc=CREG12.L75-FUT08-BUOY_clim/CREG12.L75-FUT08_m$(echo $ffileBcTMP | rev | cut -d'm' -f-2 | rev)
        ffileBm=CREG12.L75-FUT08-BUOY_annual-longtermMEAN/CREG12.L75-FUT08_y${fyyyy}_BUOY_altMEAN.nc;
        ffileBp=CREG12.L75-FUT08-BUOY_prime/${ffileBpTMP};
        ncrename -O -v cv_bn2,cv_bn2CLIM ${ffileBc} ftmpB.nc
        ncks -A -v cv_bn2AM -4 ${ffileBm} ftmpB.nc
        ncks -A -v cv_bn2 -4 ${ffileB} ftmpB.nc
        ncap2 -O -4 -s 'cv_bn2PRIME=cv_bn2-(cv_bn2AM+cv_bn2CLIM)' ftmpB.nc ftmpB.nc
        ncks -O -4 -v cv_bn2PRIME ftmpB.nc ftmpB.nc
        ncrename -O -v cv_bn2PRIME,cv_bn2 ftmpB.nc ${ffileBp}
    done; 
done
```

## Now compute the remaining climatological cycles
Here we compute the climatological seasonal cycles of total $\text{KE}$, sea ice concentration and thickness (`siconc`, `sithic`), temperature, salinity, $\sigma_{0}$ and mixed layer depth (`votemper`, `vosaline`, `rhop_sig0`, `mldr10_1`), $\text{EKE}$, total, atmosphere-ocean and ice-ocean stresses (`sozotaux`, `utau_atmoce`, `utau_iceoce`, etc.) and sea surface height `ssh`.  

```
cd $WORK/
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step};
    ncra -O -4 -v voke -d time_counter,${fivedstep},,73 CREG12.L75-REF08-KE/????/CREG12.L75-REF08_y*.5d_KE.nc CREG12.L75-REF08-KE/clim/K_tmp${step}.nc
    ncra -O -4 -v siconc,sithic -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_icemod.nc CREG12.L75-REF08-icemod_clim/I_tmp${step}.nc
    ncra -O -4 -v votemper,vosaline,rhop_sig0,mldr10_1 -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridT.nc CREG12.L75-REF08-T_clim/T_tmp${step}.nc
    ncra -O -4 -v voeke -d time_counter,${fivedstep},,73 CREG12.L75-REF08-EKE/????/CREG12.L75-REF08_y*.5d_EKE.nc CREG12.L75-REF08-EKE/clim/E_tmp${step}.nc
    ncra -O -4 -v sozotaux,utau_atmoce,utau_iceoce,uwspd10 -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridU.nc CREG12.L75-REF08-UV_clim/U_tmp${step}.nc
    ncra -O -4 -v sometauy,vtau_atmoce,vtau_iceoce,vwspd10 -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridV.nc CREG12.L75-REF08-UV_clim/V_tmp${step}.nc
    ncra -O -4 -v ssh -d time_counter,${fivedstep},,73 $REF08/????/CREG12.L75-REF08_y*.5d_gridT.nc CREG12.L75-REF08-SSH_clim/S_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-RI/1996/CREG12.L75-REF08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    Ktmp=$(echo ${tmp/RI/KEclim})
    Itmp=$(echo ${tmp/RI/icemodclim})
    Ttmp=$(echo ${tmp/RI/gridTclim})
    Etmp=$(echo ${tmp/RI/EKEclim})
    Utmp=$(echo ${tmp/RI/U2Dclim})
    Vtmp=$(echo ${tmp/RI/V2Dclim})
    Stmp=$(echo ${tmp/RI/SSHclim})
    fileK=${Ktmp/y1996/}
    fileI=${Itmp/y1996/}
    fileT=${Ttmp/y1996/}
    fileE=${Etmp/y1996/}
    fileU=${Utmp/y1996/}
    fileV=${Vtmp/y1996/}
    fileS=${Stmp/y1996/}
    mv CREG12.L75-REF08-KE/clim/K_tmp${K}.nc CREG12.L75-REF08-KE/clim/${fileK}
    mv CREG12.L75-REF08-icemod_clim/I_tmp${K}.nc CREG12.L75-REF08-icemod_clim/${fileI}
    mv CREG12.L75-REF08-T_clim/T_tmp${K}.nc CREG12.L75-REF08-T_clim/${fileT}
    mv CREG12.L75-REF08-EKE/clim/E_tmp${K}.nc CREG12.L75-REF08-EKE/clim/${fileE}
    mv CREG12.L75-REF08-UV_clim/U_tmp${K}.nc CREG12.L75-REF08-UV_clim/${fileU}
    mv CREG12.L75-REF08-UV_clim/V_tmp${K}.nc CREG12.L75-REF08-UV_clim/${fileV}
    mv CREG12.L75-REF08-SSH_clim/S_tmp${K}.nc CREG12.L75-REF08-SSH_clim/${fileS}
    k=$(($k+1))
done

cd $WORK/
for fivedstep in {0..72}; 
    do step=`printf "%02d" ${fivedstep}`
    echo ${step};
    ncra -O -4 -v voke -d time_counter,${fivedstep},,73 CREG12.L75-FUT08-KE/????/CREG12.L75-FUT08_y*.5d_KE.nc CREG12.L75-FUT08-KE/clim/K_tmp${step}.nc
    ncra -O -4 -v siconc,sithic -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_icemod.nc CREG12.L75-FUT08-icemod_clim/I_tmp${step}.nc
    ncra -O -4 -v votemper,vosaline,rhop_sig0,mldr10_1 -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridT.nc CREG12.L75-FUT08-T_clim/T_tmp${step}.nc
    ncra -O -4 -v voeke -d time_counter,${fivedstep},,73 CREG12.L75-FUT08-EKE/????/CREG12.L75FUTF08_y*.5d_EKE.nc CREG12.L75-FUT08-EKE/clim/E_tmp${step}.nc
    ncra -O -4 -v sozotaux,utau_atmoce,utau_iceoce,uwspd10 -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridU.nc CREG12.L75-FUT08-UV_clim/U_tmp${step}.nc
    ncra -O -4 -v sometauy,vtau_atmoce,vtau_iceoce,vwspd10 -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridV.nc CREG12.L75-FUT08-UV_clim/V_tmp${step}.nc
    ncra -O -4 -v ssh -d time_counter,${fivedstep},,73 $FUT08/????/CREG12.L75-FUT08_y*.5d_gridT.nc CREG12.L75-FUT08-SSH_clim/S_tmp${step}.nc
done

k=0
for fileR in /data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-RI/2051/CREG12.L75-FUT08_y*.5d_RI.nc;
    do K=`printf "%02d" ${k}`
    tmp=$(echo ${fileR} | rev | cut -d'/' -f-1 | rev)
    Ktmp=$(echo ${tmp/RI/KEclim})
    Itmp=$(echo ${tmp/RI/icemodclim})
    Ttmp=$(echo ${tmp/RI/gridTclim})
    Etmp=$(echo ${tmp/RI/EKEclim})
    Utmp=$(echo ${tmp/RI/U2Dclim})
    Vtmp=$(echo ${tmp/RI/V2Dclim})
    Stmp=$(echo ${tmp/RI/SSHclim})
    fileK=${Ktmp/y2051/}
    fileI=${Itmp/y2051/}
    fileT=${Ttmp/y2051/}
    fileE=${Etmp/y2051/}
    fileU=${Utmp/y2051/}
    fileV=${Vtmp/y2051/}
    fileS=${Stmp/y2051/}
    mv CREG12.L75-FUT08-KE/clim/K_tmp${K}.nc CREG12.L75-FUT08-KE/clim/${fileK}
    mv CREG12.L75-FUT08-icemod_clim/I_tmp${K}.nc CREG12.L75-FUT08-icemod_clim/${fileI}
    mv CREG12.L75-FUT08-T_clim/T_tmp${K}.nc CREG12.L75-FUT08-T_clim/${fileT}
    mv CREG12.L75-FUT08-EKE/clim/E_tmp${K}.nc CREG12.L75-FUT08-EKE/clim/${fileE}
    mv CREG12.L75-FUT08-UV_clim/U_tmp${K}.nc CREG12.L75-FUT08-UV_clim/${fileU}
    mv CREG12.L75-FUT08-UV_clim/V_tmp${K}.nc CREG12.L75-FUT08-UV_clim/${fileV}
    mv CREG12.L75-FUT08-SSH_clim/S_tmp${K}.nc CREG12.L75-FUT08-SSH_clim/${fileS}
    k=$(($k+1))
done
```

## Calculate the Brunt-V&auml;is&auml;l&auml; frequency N$^{2}$

```
cd $WORK
for yyyy in {1996..2015}; do     
    echo ${yyyy};     
    for fileT in $REF08/${yyyy}/CREG12.L75-REF08_y${yyyy}*.5d_gridT.nc; do        
        fileN=$(echo ${fileT/gridT/Nsquared} | rev | cut -d'/' -f-2 | rev);                 
        /home/rieck/src/CDFTOOLS/bin/cdfbn2 -t ${fileT} -nc4 -o $WORK/CREG12.L75-REF08-Nsquared/${fileN};              
    done; 
done
for fileT in CREG12.L75-REF08-T_clim/CREG12.L75-REF08_m* ; do
    fileN=$(echo ${fileT/gridTclim/Nsquared} | rev | cut -d'/' -f-1 | rev);                 
    /home/rieck/src/CDFTOOLS/bin/cdfbn2 -t ${fileT} -nc4 -o $WORK/CREG12.L75-REF08-Nsquared/clim/${fileN}
done

cd $WORK
for yyyy in {2051..2070}; do  
    echo ${yyyy}
    for fileT in $FUT08/${yyyy}/CREG12.L75-FUT08_y${yyyy}*.5d_gridT.nc; do        
        fileN=$(echo ${fileT/gridT/Nsquared} | rev | cut -d'/' -f-2 | rev);                 
        /home/rieck/src/CDFTOOLS/bin/cdfbn2 -t ${fileT} -nc4 -o $WORK/CREG12.L75-FUT08-Nsquared/${fileN};              
    done;  
done
for fileT in CREG12.L75-FUT08-T_clim/CREG12.L75-FUT08_m* ; do
    fileN=$(echo ${fileT/gridTclim/Nsquared} | rev | cut -d'/' -f-1 | rev);                 
    /home/rieck/src/CDFTOOLS/bin/cdfbn2 -t ${fileT} -nc4 -o $WORK/CREG12.L75-FUT08-Nsquared/clim/${fileN}
done
```